In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.7-py2.py3-none-any.whl size=317907716 sha256=79b009a5813cdc7296bae2d43131d2d7bdc7e62166a7b0b59d121bc5d167b543
  Stored in directory: /opt/work/.cache/pip/wheels/90/7b/3a/f4e0b629a2ec0ac23c6ea92ee30d7fb269def367fd9f70bcf9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [6]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST avito")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.rest.type", "rest")
    .config("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.rest.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.rest.s3.path-style-access", "true")
    .config("spark.sql.catalog.rest.s3.access-key-id", "admin")
    .config("spark.sql.catalog.rest.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.rest.s3.region", "us-east-1")
    .getOrCreate()
)

spark

In [7]:
spark.sql("SHOW NAMESPACES IN rest").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
|silver   |
+---------+



In [8]:
spark.sql("SHOW TABLES IN rest.raw").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|raw      |avito    |false      |
|raw      |mubawab  |false      |
+---------+---------+-----------+



In [9]:
# park.sql("TRUNCATE TABLE rest.raw.avito")

In [11]:
spark.sql("SELECT COUNT(*) FROM rest.raw.avito").show()

+--------+
|count(1)|
+--------+
|      74|
+--------+



In [94]:
# Load RAW table
raw_avito = spark.table("rest.raw.avito")   # or local.raw.sarouty

In [95]:
import pandas as pd

pd.set_option('display.max_colwidth', None)  # show full column text
pd.set_option('display.width', None)         # no wrapping
pd.set_option('display.max_rows', 10)        # adjust as you like

raw_avito.limit(1).toPandas()

,id,payload,ingest_ts
0,57017784,"{""id"": ""57017784"", ""url"": ""https://www.avito.ma/fr/bir_rami/appartements/Appartement_à_louer_74_m²_à_Kénitra_57017784.htm"", ""error"": null, ""listing_type"": ""location"", ""title"": ""Appartement à louer 74 m² à Kénitra"", ""price_text"": ""6 500 DH"", ""breadcrumbs"": ""Home Icon Accueil > Tout le Maroc > Kénitra > Bir Rami > Avito Immobilier > Locations Immobilières > Appartements > Appartement à louer 74 m² à Kénitra"", ""category"": ""Appartements, à louer"", ""description"": ""Offre de Location : Charmant Appartement Meublé à Bir Rami Coup de cœur assuré À louer, un magnifique appartement entièrement meublé de 77 m², idéalement situé au cœur du quartier de Bir Rami. Cet appartement se trouve au 2ème étage d'une résidence moderne et très bien entretenue, entourée d'un jardin joliment aménagé, offrant un cadre de vie calme et agréable. Composition et Prestations • Un salon confortable et une salle à manger conviviale. • Deux (2) chambres lumineuses. • Une cuisine moderne entièrement équipée. • Une buanderie pratique. • Une salle d'eau et une salle de bain avec douche. • Inclus : une place de parking privative. Conditions de Location : • Loyer mensuel : 6.500,00 DH (Char Parfait pour un couple ou un professionnel à la recherche de confort, de commodité et d'un environnement soigné. Prêt à emménager immédiatement Contactez nous sans tarder pour organiser une visite et saisir cette opportunité"", ""attributes"": ""{\""Chambres\"": \""2\"", \""Salle de bain\"": \""1\"", \""Surface totale\"": \""77\"", \""Caution\"": \""1 mois\"", \""Salons\"": \""2\"", \""Surface habitable\"": \""74\""}"", ""equipments"": ""2; 1; 77; 1 mois; 2; 74; Ascenseur; Concierge; Cuisine équipée; Parking; Sécurité"", ""seller_name"": ""injidab immo"", ""seller_type"": ""Particulier"", ""published_date"": ""2025-11-07 17:37:40"", ""image_urls"": ""https://content.avito.ma/classifieds/images/10140629219?t=images | https://content.avito.ma/classifieds/images/10140629225?t=images | https://content.avito.ma/classifieds/images/10140629230?t=images | https://content.avito.ma/classifieds/images/10140629236?t=images | https://content.avito.ma/classifieds/images/10140629239?t=images | https://content.avito.ma/classifieds/images/10140629243?t=images | https://content.avito.ma/classifieds/images/10140629244?t=images | https://content.avito.ma/classifieds/images/10140629245?t=images | https://content.avito.ma/classifieds/images/10140629252?t=images | https://content.avito.ma/classifieds/images/10140629253?t=images | https://content.avito.ma/classifieds/images/10140629266?t=images | https://content.avito.ma/classifieds/images/10140629254?t=images | https://content.avito.ma/classifieds/images/10140629265?t=images | https://content.avito.ma/classifieds/images/10140629261?t=images""}",2025-11-07 16:58:50.010


In [96]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

# Define the JSON schema inside "payload"
payload_schema = StructType([
    StructField("id", StringType()),
    StructField("url", StringType()),
    StructField("error", StringType()),
    StructField("title", StringType()),
    StructField("price_text", StringType()),
    StructField("breadcrumbs", StringType()),
    StructField("category", StringType()),
    StructField("description", StringType()),
    StructField("attributes", StringType()),
    StructField("equipments", StringType()),
    StructField("seller_name", StringType()),
    StructField("seller_type", StringType()),
    StructField("published_date", StringType()),
    StructField("image_urls", StringType()),
    StructField("listing_type", StringType()),   # ✅ NEW COLUMN
])

raw_avito = (
    raw_avito
    .withColumn("json", from_json(col("payload"), payload_schema))
    .select(
        col("id").alias("record_id"),
        "json.*",   # expand all JSON fields
        "ingest_ts"
    )
)


In [97]:

pd.set_option('display.max_colwidth', 30)  # show full column text
pd.set_option('display.width', 20)         # no wrapping
pd.set_option('display.max_rows', 15)  

raw_avito.limit(1).toPandas()

,record_id,id,url,error,title,price_text,breadcrumbs,category,description,attributes,equipments,seller_name,seller_type,published_date,image_urls,listing_type,ingest_ts
0,54069528,54069528,https://www.avito.ma/fr/sa...,None,Villas titrés façade Rte K...,4 500 DH,Home Icon Accueil > Tout l...,"Terrains et fermes, à vendre",Lots de villas de 250 à 40...,"{""Surface totale"": ""250"", ...",250; Villa; Loti; Titré,adil,Particulier,2025-11-07 18:04:10,https://content.avito.ma/c...,vente,2025-11-07 18:04:20.004


## id

In [98]:
raw_avito.select("id", "record_id").filter(col("id") != col("record_id")).show(truncate=False)

[Stage 9:=====================================================> (221 + 6) / 227]

+---+---------+
|id |record_id|
+---+---------+
+---+---------+



In [99]:
raw_avito = raw_avito.drop("record_id")

In [100]:
from pyspark.sql.functions import col, count


dupes = (
    raw_avito.groupBy("id")
      .agg(count("*").alias("count"))
      .filter(col("count") > 1)
      .orderBy(col("count").desc())
)

dupes.show(truncate=False)
print("Total duplicate IDs:", dupes.count())

+--------+-----+
|id      |count|
+--------+-----+
|57050520|3    |
|57166662|3    |
|57170838|3    |
|56255040|3    |
|56663398|3    |
|57017784|3    |
|57170675|3    |
|38583363|3    |
|56975025|3    |
|57130608|3    |
|55856750|3    |
|57162000|3    |
|57029324|3    |
|56690395|3    |
|57109849|3    |
|57110660|3    |
|56960539|3    |
|54938415|3    |
|57142186|3    |
|57171586|3    |
+--------+-----+
only showing top 20 rows



[Stage 13:=====================================================>(346 + 6) / 352]

Total duplicate IDs: 3068


In [101]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number


# Window: group by id, order by most recent ingest_ts
w = Window.partitionBy("id").orderBy(col("ingest_ts").desc())

# Keep only first row (latest per id)
raw_avito = (
    raw_avito.withColumn("rn", row_number().over(w))
      .filter(col("rn") == 1)
      .drop("rn")
)

## url

In [102]:
from pyspark.sql.functions import col, trim

raw_avito = raw_avito.filter(
    (col("url").isNotNull()) &
    (trim(col("url")) != "")
)

## price 

In [103]:
raw_avito.select("id","price_text").show(10, truncate=False)

[Stage 19:==================================================>   (332 + 6) / 352]

+--------+-------------+
|id      |price_text   |
+--------+-------------+
|29787804|170 000 DH   |
|29788248|1 764 000 DH |
|29788451|750 000 DH   |
|29804839|504 000 DH   |
|29806108|504 000 DH   |
|29806631|1 370 000 DH |
|30766390|1 300 000 DH |
|30911984|96 000 000 DH|
|31087743|4 500 DH     |
|31752899|594 000 DH   |
+--------+-------------+
only showing top 10 rows



In [104]:
from pyspark.sql.functions import col, length

null_price_text = raw_avito.filter(
    col("price_text").isNull() | (length(col("price_text")) == 0)
).count()

print(f"Rows with NULL or empty price_text: {null_price_text}")

[Stage 22:===================================================>  (336 + 6) / 352]

Rows with NULL or empty price_text: 604


In [105]:
from pyspark.sql.functions import col, when, regexp_replace, length, lit

raw_avito = (
    raw_avito
    # Nettoyage du texte et conversion en nombre
    .withColumn(
        "price",
        when(
            (col("price_text").isNull()) | (length(col("price_text")) == 0),
            lit(None).cast("double")
        ).otherwise(
            regexp_replace(  # supprime caractères inutiles et convertit en double
                regexp_replace(
                    regexp_replace(col("price_text"), u"\u00A0", ""),  # supprime NBSP
                    r"[ ,]", ""                                       # supprime espaces et virgules
                ),
                r"[^0-9.]", ""                                       # garde uniquement chiffres et points
            ).cast("double")
        )
    )
    .drop("price_text")  # supprime l'ancienne colonne
)

raw_avito.select("id", "price").show(10, truncate=False)
null_price = raw_avito.filter(col("price").isNull()).count()
print(f"Rows with NULL price after cleaning: {null_price}")

+--------+---------+
|id      |price    |
+--------+---------+
|29787804|170000.0 |
|29788248|1764000.0|
|29788451|750000.0 |
|29804839|504000.0 |
|29806108|504000.0 |
|29806631|1370000.0|
|30766390|1300000.0|
|30911984|9.6E7    |
|31087743|4500.0   |
|31752899|594000.0 |
+--------+---------+
only showing top 10 rows



[Stage 31:=====================================================>(351 + 1) / 352]

Rows with NULL price after cleaning: 604


In [106]:
from pyspark.sql.functions import col, format_number, regexp_replace
from pyspark.sql.types import DecimalType

raw_avito.select(
    "id",
    # cast to high-precision decimal → format without decimals → drop commas
    regexp_replace(
        format_number(col("price").cast(DecimalType(38, 0)), 0),
        ",", ""
    ).alias("price")
).show(10, truncate=False)

[Stage 37:=====================================================>(350 + 2) / 352]

+--------+--------+
|id      |price   |
+--------+--------+
|29787804|170000  |
|29788248|1764000 |
|29788451|750000  |
|29804839|504000  |
|29806108|504000  |
|29806631|1370000 |
|30766390|1300000 |
|30911984|96000000|
|31087743|4500    |
|31752899|594000  |
+--------+--------+
only showing top 10 rows



## seller

In [71]:
raw_avito.select("id","seller_name" , "seller_type").show(10, truncate=False)

[Stage 195:===================================================> (314 + 6) / 324]

+--------+-----------------+-----------+
|id      |seller_name      |seller_type|
+--------+-----------------+-----------+
|29787804|CHAABI LIL ISKANE|Particulier|
|29788248|CHAABI LIL ISKANE|Particulier|
|29788451|CHAABI LIL ISKANE|Particulier|
|29804839|CHAABI LIL ISKANE|Particulier|
|29806108|CHAABI LIL ISKANE|Particulier|
|29806631|CHAABI LIL ISKANE|Particulier|
|30766390|promozin         |Particulier|
|30911984|promozin         |Particulier|
|31087743|promozin         |Particulier|
|31752899|CHAABI LIL ISKANE|Particulier|
+--------+-----------------+-----------+
only showing top 10 rows



In [107]:
from pyspark.sql.functions import col, count

raw_avito.groupBy("seller_type") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(truncate=False)

[Stage 40:=====================================================>(349 + 3) / 352]

+-----------+-----+
|seller_type|count|
+-----------+-----+
|Particulier|4194 |
|NULL       |6    |
+-----------+-----+



In [108]:
raw_avito = raw_avito.drop("seller_type")

In [74]:
raw_avito.groupBy("seller_name") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

[Stage 204:==================================================>  (311 + 6) / 324]

+----------------------------+-----+
|seller_name                 |count|
+----------------------------+-----+
|Serraj Immobilier           |107  |
|Between bonnes affaires immo|90   |
|CHAABI LIL ISKANE           |71   |
|Ouchra Immobilier           |57   |
|Logic Immo                  |55   |
|injidab immo                |52   |
|Serraj immobilier           |50   |
|LE 25 IMMOBILIER            |42   |
|ANOUAR IMMO                 |38   |
|Agence Immobliere ABOU ISHAK|38   |
+----------------------------+-----+
only showing top 10 rows



In [109]:
from pyspark.sql.functions import col, lower, when, trim

raw_avito = (
    raw_avito
    .withColumn(
        "seller",
        when(
            (col("seller_name").isNull()) |
            (trim(col("seller_name")) == "") |
            (lower(trim(col("seller_name"))).isin("null", "none", "unknown")),
            "unknown"
        ).otherwise(lower(trim(col("seller_name"))))
    )
    .drop("seller_name")
)

## images

In [110]:
from pyspark.sql.functions import expr

raw_avito = (
    raw_avito
    .withColumn(
        "images",
        expr(
            "FILTER(TRANSFORM(SPLIT(image_urls, '\\\\s*\\\\|\\\\s*'), x -> TRIM(x)), x -> x <> '')"
        )
    )
    .drop("image_urls")
)


In [25]:
raw_avito.select("images").distinct().show(1, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|images                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

## equipments

In [77]:
raw_avito.groupBy("equipments") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(5, truncate=False)

[Stage 210:====================================================>(318 + 6) / 324]

+--------------------+-----+
|equipments          |count|
+--------------------+-----+
|NULL                |151  |
|3; 1; 120; 1; 1; 120|29   |
|3; 1; 120; 1; 120; 1|29   |
|Maison; Loti        |12   |
|Maison              |11   |
+--------------------+-----+
only showing top 5 rows



In [111]:
from pyspark.sql.functions import expr, lower, trim

raw_avito = raw_avito.withColumn(
    "equipments",
    expr("""
      CASE
        WHEN equipments IS NULL THEN array()
        ELSE array_distinct(
          FILTER(
            TRANSFORM(
              SPLIT(equipments, '\\s*;\\s*'),
              x -> trim(x)
            ),
            x ->
              x <> '' AND
              x RLIKE '.*[A-Za-zÀ-ÿ].*' AND
              NOT (x RLIKE '.*[0-9].*') AND           -- remove any digits (1, 2, 1000…)
              NOT (lower(x) RLIKE '.*moi.*') AND      -- remove 'moi', 'mois', '1 moi'…
              NOT (lower(x) RLIKE '^(aucune|studio)$')
          )
        )
      END
    """)
)

In [79]:
raw_avito.groupBy("equipments") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

[Stage 216:====================================================>(320 + 4) / 324]

+------------------------------------------------------------------------------+-----+
|equipments                                                                    |count|
+------------------------------------------------------------------------------+-----+
|[]                                                                            |994  |
|[Ascenseur, Balcon, Chauffage, Climatisation, Concierge, Cuisine équipée]     |250  |
|[Neuf, Immédiate]                                                             |73   |
|[Ascenseur, Balcon, Climatisation, Concierge, Cuisine équipée, Meublé]        |71   |
|[Agricole, Titré]                                                             |68   |
|[Bon état, Immédiate]                                                         |67   |
|[Ascenseur, Chauffage, Climatisation, Câblage téléphonique, Parking, Sécurité]|66   |
|[Neuf]                                                                        |53   |
|[Balcon, Chauffage, Climatisation, Cuisine

In [80]:
raw_avito.limit(1).toPandas()

,id,url,error,title,breadcrumbs,category,description,attributes,equipments,published_date,listing_type,ingest_ts,price,seller,images
0,43321433,https://www.avito.ma/fr/au...,None,Terrain Industriel de 8650...,Home Icon Accueil > Tout l...,"Terrains et fermes, à vendre",Un terrain Industriel à Ti...,"{""Surface totale"": ""8650"",...","[Industriel, Titré]",2025-11-07 14:10:25,vente,2025-11-07 20:10:30.002,6487500.0,ouchra immobilier,[https://content.avito.ma/...


In [81]:
raw_avito.groupBy("category") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(50, truncate=False)

[Stage 225:==============================================>      (282 + 6) / 324]

+----------------------------+-----+
|category                    |count|
+----------------------------+-----+
|Appartements, à louer       |1384 |
|Appartements, à vendre      |972  |
|Terrains et fermes, à vendre|431  |
|Villas et Riads, à vendre   |336  |
|Bureaux, à louer            |295  |
|Local, à louer              |248  |
|Local, à vendre             |149  |
|Villas et Riads, à louer    |113  |
|Maisons, à vendre           |98   |
|Bureaux, à vendre           |58   |
|Autre Immobilier, à vendre  |49   |
|Maisons, à louer            |35   |
|Autre Immobilier, à louer   |19   |
|NULL                        |7    |
|Terrains et fermes, à louer |5    |
+----------------------------+-----+



## offre 

In [82]:
raw_avito.groupBy("listing_type") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(50, truncate=False)

[Stage 231:====================================================>(320 + 4) / 324]

+------------+-----+
|listing_type|count|
+------------+-----+
|location    |2104 |
|vente       |2095 |
+------------+-----+



In [112]:
from pyspark.sql.functions import col

raw_avito = raw_avito.withColumnRenamed("listing_type", "offre")

## city & nighbrhood

In [84]:
raw_avito.groupBy("breadcrumbs") \
    .agg(count("*").alias("count")) \
    .orderBy(col("count").desc()) \
    .show(50, truncate=False)

[Stage 237:======================================>              (234 + 6) / 324]

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|breadcrumbs                                                                                                                                                                     |count|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|NULL                                                                                                                                                                            |7    |
|Home Icon Accueil > Tout le Maroc > Mohammedia > Mimosas > Avito Immobilier > Locations Immobilières > Appartements > Appartement à louer 80 m² à Mohammedia                    |6    |
|Home Icon Accueil > Tout le Maroc > Martil > Toute la ville > Avito Immobi

In [113]:
from pyspark.sql.functions import (
    split, trim, col, when, size, array_position, element_at, coalesce, lit
)

# Split breadcrumbs safely
parts = split(coalesce(col("breadcrumbs"), lit("")), " > ")

# 1) Anchor index of "Tout le Maroc"
#    NOTE: pass a plain string, not lit("..."), to array_position
idx = array_position(parts, "Tout le Maroc")   # returns 1-based index; 0 if not found

# 2) Extract city and neighborhood relative to the anchor
raw_avito = (
    raw_avito
    .withColumn(
        "city",
        when(
            (idx > lit(0)) & (size(parts) >= (idx + lit(1))),
            trim(element_at(parts, (idx + lit(1)).cast("int")))
        )
    )
    .withColumn(
        "neighborhood_raw",
        when(
            (idx > lit(0)) & (size(parts) >= (idx + lit(2))),
            trim(element_at(parts, (idx + lit(2)).cast("int")))
        )
    )
)

# 3) Clean neighborhood: drop Avito category labels and generic placeholders
bad_neigh = ["Avito Immobilier", "أفيتو للعقار", "Toute la ville", "Autre secteur"]
raw_avito = (
    raw_avito
    .withColumn(
        "neighborhood",
        when(col("neighborhood_raw").isin(bad_neigh), None).otherwise(col("neighborhood_raw"))
    )
    .drop("neighborhood_raw")
)

# Check
raw_avito.select("city", "neighborhood").show(50, truncate=False)

[Stage 46:===================================================>  (333 + 6) / 352]

+--------------+-------------------------+
|city          |neighborhood             |
+--------------+-------------------------+
|Tanger        |Drissia                  |
|Tanger        |Centre ville             |
|Marrakech     |Rouidat                  |
|Marrakech     |Allal El Fassi           |
|Casablanca    |Californie               |
|Marrakech     |Route de Lalla Takerkoust|
|Kénitra       |Bir Rami Est             |
|Kénitra       |Val Fleury               |
|Casablanca    |Sidi Bernoussi           |
|Sidi Bouknadel|NULL                     |
|Rabat         |Hay el Fath              |
|Mohammedia    |NULL                     |
|Kénitra       |Fouarate                 |
|Kénitra       |Maamora                  |
|Kénitra       |Bir Rami                 |
|Casablanca    |Sidi Maarouf             |
|Marrakech     |Guéliz                   |
|Salé          |Tabriquet                |
|Casablanca    |Route d'El Jadida        |
|Temara        |NULL                     |
|Kénitra   

In [114]:
from pyspark.sql.functions import lit

raw_avito = raw_avito.withColumn("site", lit("avito"))

In [115]:
raw_avito.select("category").show(5, truncate=False)

[Stage 49:==================================================>   (332 + 6) / 352]

+----------------------+
|category              |
+----------------------+
|Appartements, à vendre|
|Bureaux, à vendre     |
|Appartements, à vendre|
|Appartements, à vendre|
|Appartements, à vendre|
+----------------------+
only showing top 5 rows



In [116]:
from pyspark.sql.functions import split, trim, col, when, coalesce, lit, size

# Split on comma with optional spaces around it
parts = split(coalesce(col("category"), lit("")), r"\s*,\s*")

raw_avito = (
    raw_avito
    .withColumn("property_type", when(size(parts) >= 1, trim(parts.getItem(0))))
    .withColumn("listing_phrase", when(size(parts) >= 2, trim(parts.getItem(1))))
)

# Quick check:
raw_avito.select("category", "property_type", "listing_phrase").show(50, truncate=False)

[Stage 52:====================================================> (341 + 6) / 352]

+----------------------------+------------------+--------------+
|category                    |property_type     |listing_phrase|
+----------------------------+------------------+--------------+
|Appartements, à vendre      |Appartements      |à vendre      |
|Bureaux, à vendre           |Bureaux           |à vendre      |
|Appartements, à vendre      |Appartements      |à vendre      |
|Appartements, à vendre      |Appartements      |à vendre      |
|Appartements, à vendre      |Appartements      |à vendre      |
|Appartements, à vendre      |Appartements      |à vendre      |
|Local, à louer              |Local             |à louer       |
|Appartements, à vendre      |Appartements      |à vendre      |
|Appartements, à louer       |Appartements      |à louer       |
|Appartements, à vendre      |Appartements      |à vendre      |
|Autre Immobilier, à louer   |Autre Immobilier  |à louer       |
|Appartements, à louer       |Appartements      |à louer       |
|Appartements, à louer   

In [117]:
from pyspark.sql.functions import when, col, lit

# Map French phrase → normalized form
raw_avito = (
    raw_avito
    .withColumn(
        "listing_expected",
        when(col("listing_phrase") == "à louer", lit("location"))
        .when(col("listing_phrase") == "a louer", lit("location"))
        .when(col("listing_phrase") == "à vendre", lit("vente"))
        .when(col("listing_phrase") == "a vendre", lit("vente"))
        .otherwise(lit(None))
    )
    # Compare the mapped form to the 'offre' column
    .withColumn(
        "offre_match",
        when(col("listing_expected") == col("offre"), lit(True)).otherwise(lit(False))
    )
)

# Optional: see mismatches
raw_avito.filter(~col("offre_match")).select("category", "listing_phrase", "offre", "listing_expected").show(50, truncate=False)

[Stage 55:===================================================>  (334 + 6) / 352]

+--------+--------------+--------+----------------+
|category|listing_phrase|offre   |listing_expected|
+--------+--------------+--------+----------------+
|NULL    |NULL          |location|NULL            |
|NULL    |NULL          |location|NULL            |
|NULL    |NULL          |vente   |NULL            |
|NULL    |NULL          |location|NULL            |
|NULL    |NULL          |location|NULL            |
|NULL    |NULL          |vente   |NULL            |
+--------+--------------+--------+----------------+



In [118]:
raw_avito = raw_avito.drop("category", "listing_phrase", "listing_expected")

In [48]:
raw_avito.select("attributes").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------+
|attributes                                                                                                           |
+---------------------------------------------------------------------------------------------------------------------+
|{"Surface totale": "67", "Nombre de pièces": "3", "Étage": "2"}                                                      |
|{"Salle de bain": "3", "Surface totale": "12000"}                                                                    |
|{"Chambres": "2", "Salle de bain": "1", "Caution": "1 mois", "Salons": "1", "Surface habitable": "100", "Étage": "0"}|
|{"Nombre de pièces": "1", "Étage": "1"}                                                                              |
|{"Salle de bain": "2", "Surface totale": "120", "Nombre de pièces": "5", "Étage": "2"}                               |
+---------------------------------------

In [119]:
from pyspark.sql.functions import from_json, col, trim, expr
from pyspark.sql.types import MapType, StringType
import unicodedata, re

# 1) Parse JSON → Map(String,String)
raw_avito = raw_avito.withColumn(
    "attr_map",
    from_json(col("attributes"), MapType(StringType(), StringType()))
)

# 2) Collect ALL distinct keys present in the map
keys_df = raw_avito.selectExpr("explode(map_keys(attr_map)) as k").distinct()
keys = [r["k"] for r in keys_df.collect() if r["k"] is not None]

# 3) Helper: sanitize keys → safe Spark column names (snake_case, ASCII)
def sanitize(name: str) -> str:
    # remove accents
    name = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("ascii")
    # to snake_case
    name = re.sub(r"[^0-9a-zA-Z]+", "_", name).strip("_")
    return name.lower()

# 4) Create a column per key dynamically
for k in keys:
    raw_avito = raw_avito.withColumn(sanitize(k), trim(col("attr_map")[k]))

# 5) Optional: drop the intermediate map column
raw_avito = raw_avito.drop("attr_map")

# (Optional) Quick peek:
# raw_avito.select("attributes", *[sanitize(k) for k in keys]).show(20, truncate=False)


In [120]:
raw_avito.select("attributes", *[sanitize(k) for k in keys]).show(20, truncate=False)

[Stage 64:===================================================>  (336 + 6) / 352]

+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-------+----------+------------------+----------+--------------+-----+-------------+----------------+--------+-------------+--------------------+---------+--------------+-------------+------+
|attributes                                                                                                                                          |surface_habitable|caution|zoning    |type_d_appartement|standing  |surface_totale|etage|age_du_bien  |nombre_de_pieces|chambres|salle_de_bain|frais_de_syndic_mois|condition|nombre_d_etage|disponibilite|salons|
+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-------+----------+------------------+----------+--------------+-----+-------------+----------------+--------+--

In [121]:
raw_avito.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- error: string (nullable = true)
 |-- title: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- description: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- published_date: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- seller: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- site: string (nullable = false)
 |-- property_type: string (nullable = true)
 |-- offre_match: boolean (nullable = false)
 |-- surface_habitable: string (nullable = true)
 |-- caution: string (nullable = true)
 |-- zoning: string (nullable = true)
 |-- type_d_a

In [51]:
raw_avito.limit(1).toPandas()

,id,url,error,title,breadcrumbs,description,attributes,equipments,published_date,offre,...,caution,type_d_appartement,surface_totale,etage,nombre_de_pieces,salle_de_bain,chambres,frais_de_syndic_mois,nombre_d_etage,salons
0,38583363,https://www.avito.ma/fr/gu...,None,Bureau de 67 m2 à louer à ...,Home Icon Accueil > Tout l...,On vous propose un très jo...,"{""Surface totale"": ""67"", ""...","[Air Conditioning, Elevato...",2025-11-07 17:40:38,location,...,None,None,67,2,3,None,None,None,None,None
